In [20]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr)

In [21]:
##Crear folders de almacenamiento
dir.create("microdatos", showWarnings = F)

In [22]:
# Función para descargar y descomprimir un archivo con timeout
download_and_unzip <- function(url, destination="microdatos") {
  tryCatch({
    # Descargar el archivo con timeout
    download.file(url, destfile = paste0(destination, "/", basename(url)), mode = "wb", timeout = 10000)

    # Descomprimir el archivo
    unzip(paste0(destination, "/", basename(url)), exdir = destination)

    # Eliminar el archivo comprimido después de descomprimirlo
    unlink(paste0(destination, "/", basename(url)))

    message(paste("Descarga y descompresión exitosas para", basename(url)))
  }, error = function(e) {
    warning(paste("Error al descargar o descomprimir", basename(url), ":", conditionMessage(e)))
  })
}

In [23]:
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_poblacion_csv.zip")
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_trabajos_csv.zip")

Descarga y descompresión exitosas para enigh2022_ns_poblacion_csv.zip

Descarga y descompresión exitosas para enigh2022_ns_trabajos_csv.zip



In [24]:
#Leer población
poblacion <- read_csv("microdatos/poblacion.csv",show_col_types=FALSE)%>%

mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")%>%
  #Crear grupos de edad
  mutate(grupo_edad = case_when(
    edad>=0 & edad<=5 ~ "0-5",
    edad>=6 & edad<=11 ~ "6-11",
    edad>=12 & edad<=14 ~ "12-14",
    edad>=15 & edad<=17 ~ "15-17",
    edad>=18 & edad<=24 ~ "18-24",
    edad>=25 & edad<=29 ~ "25-29",
    edad>=30 & edad<=34 ~ "30-34",
    edad>=35 & edad<=39 ~ "35-39",
    edad>=40 & edad<=44 ~ "40-44",
    edad>=45 & edad<=49 ~ "45-49",
    edad>=50 & edad<=54 ~ "50-54",
    edad>=55 & edad<=59 ~ "55-59",
    edad>=60 & edad<=64 ~ "60-64",
    edad>=65 & edad<=69 ~ "65-69",
    edad>=70 & edad<=74 ~ "70-74",
    edad>=75 & edad<=79 ~ "75-79",
    edad>=80 & edad<=84 ~ "80-84",
    edad>=85 ~ "85+",
    TRUE ~ NA_character_
  )) 

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


In [25]:
trabajos<- read_csv("microdatos/trabajos.csv",show_col_types=FALSE)%>%
  mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")%>%
  select(folioviv,foliohog,numren,pres_5)%>%

    #Cambiar pres_5 a 1 si es "05" y 0 si es NA
    mutate(pres_5=case_when(
        pres_5 == "05" ~ 1,
        TRUE ~ 0
    )
    )%>%
    #Agrupar por folioviv, foliohog y numren y sumar pres_5
    group_by(folioviv, foliohog, numren) %>%
    summarise(pres_5 = sum(pres_5, na.rm = TRUE))%>%
    #Si es 1 o más, dejarlo como 1
    mutate(pres_5 = ifelse(pres_5 >= 1, 1, 0),
        #Identificador persona ocupada
    ocupado=1
    
    )
trabajos


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
`summarise()` has grouped output by 'folioviv', 'foliohog'. You can override
using the `.groups` argument.


folioviv,foliohog,numren,pres_5,ocupado
<chr>,<dbl>,<chr>,<dbl>,<dbl>
0900006901,1,02,1,1
0900006901,1,03,1,1
0900006902,1,03,1,1
0900006902,1,04,1,1
0900006903,1,01,0,1
0900006903,1,02,0,1
0900006904,1,01,1,1
0900006904,1,02,1,1
0900006905,1,02,1,1


In [26]:
ocupados<-poblacion%>%
left_join(trabajos, by=c("folioviv","foliohog","numren"))%>%
  #Dejar solo personas ocupadas
  filter(ocupado==1)

In [27]:
dm<-ocupados%>%
as_survey_design(ids=upm,weights=factor,strata=est_dis)

In [ ]:
print("Del total de jóvenes ocupados, ¿qué porcentaje recibe crédito para vivienda como prestación laboral?")
dm%>%
mutate(ident_jovenes = case_when(
    grupo_edad %in% c("12-14", "15-17", "18-24", "25-29") ~ "joven",
    TRUE ~ "otros"
  ))%>%
group_by(ident_jovenes,pres_5)%>%
summarise(ocupados = survey_total(vartype="cv"),
pct=survey_prop(vartype = "cv"))%>%
mutate(pct = round(pct * 100,1)) %>%
ungroup()%>%
#Filtrar los que no tienen pres_5
filter(pres_5==0)

[1] "Del total de jóvenes ocupados, ¿qué porcentaje recibe crédito para vivienda como prestación laboral?"


ident_jovenes,pres_5,ocupados,ocupados_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
joven,0,671326,0.05563368,72.4,0.02790521
otros,0,2333491,0.02926371,64.6,0.01932521
